In [1]:
!pip install pandas numpy

In [2]:
from google.colab import files
uploaded = files.upload()


Saving data.json to data.json


In [3]:
import json
import numpy as np
import pandas as pd
from collections import Counter
from datetime import datetime

**Scoring & Feature Functions**

In [4]:
def entropy(actions):
    counts = np.array(list(Counter(actions).values()))
    probs = counts / counts.sum()
    return -np.sum(probs * np.log2(probs)) if probs.size > 0 else 0

def compute_features(wallet, txns):
    actions = [tx["action"] for tx in txns]
    timestamps = [tx["timestamp"] for tx in txns]

    df = pd.DataFrame(txns)
    total_deposit = df[df["action"] == "deposit"]["amount"].sum()
    total_borrow = df[df["action"] == "borrow"]["amount"].sum()
    total_repay = df[df["action"] == "repay"]["amount"].sum()
    total_redeem = df[df["action"] == "redeemunderlying"]["amount"].sum()
    total_liquid = df[df["action"] == "liquidationcall"].shape[0]

    repay_ratio = total_repay / total_borrow if total_borrow > 0 else 1
    redeem_ratio = total_redeem / total_deposit if total_deposit > 0 else 0
    repay_pct = len(df[df["action"] == "repay"]) / len(df)

    unique_days = len(set([datetime.utcfromtimestamp(t).date() for t in timestamps]))
    avg_time_gap = np.mean(np.diff(sorted(timestamps))) if len(timestamps) > 1 else 0
    action_entropy = entropy(actions)

    utilization = total_borrow - total_repay
    utilization_ratio = utilization / total_borrow if total_borrow > 0 else 0

    return {
        "wallet": wallet,
        "deposit_amt": total_deposit,
        "borrow_amt": total_borrow,
        "repay_amt": total_repay,
        "repay_ratio": repay_ratio,
        "redeem_ratio": redeem_ratio,
        "repay_pct": repay_pct,
        "liquidations": total_liquid,
        "avg_time_gap": avg_time_gap,
        "active_days": unique_days,
        "action_entropy": action_entropy,
        "util_ratio": utilization_ratio,
        "total_txns": len(txns)
    }


In [5]:
def score_wallet(row):
    score = 500

    score += 100 * min(row["repay_ratio"], 1)
    score += 50 * (1 - row["util_ratio"])
    score += 50 * row["redeem_ratio"]
    score += 50 * row["repay_pct"]
    score += 30 * row["action_entropy"]
    score += 10 * min(row["active_days"], 30) / 30

    score -= 100 * row["liquidations"]
    score -= 20 * (1 / (1 + row["avg_time_gap"]))  # avoid tiny gaps

    return max(0, min(1000, int(score)))


In [8]:
with open("data.json", "r") as f:
    data = json.load(f)

print(type(data))
print(data[:2])  # Preview first 2 items


<class 'list'>
[{'_id': {'$oid': '681d38fed63812d4655f571a'}, 'userWallet': '0x00000000001accfa9cef68cf5371a23025b6d4b6', 'network': 'polygon', 'protocol': 'aave_v2', 'txHash': '0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190', 'logId': '0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190_Deposit', 'timestamp': 1629178166, 'blockNumber': 1629178166, 'action': 'deposit', 'actionData': {'type': 'Deposit', 'amount': '2000000000', 'assetSymbol': 'USDC', 'assetPriceUSD': '0.9938318274296357543568636362026045', 'poolId': '0x2791bca1f2de4661ed88a30c99a7a9449aa84174', 'userId': '0x00000000001accfa9cef68cf5371a23025b6d4b6'}, '__v': 0, 'createdAt': {'$date': '2025-05-08T23:06:39.465Z'}, 'updatedAt': {'$date': '2025-05-08T23:06:39.465Z'}}, {'_id': {'$oid': '681aa70dd6df53021cc6f3c0'}, 'userWallet': '0x000000000051d07a4fb3bd10121a343d85818da6', 'network': 'polygon', 'protocol': 'aave_v2', 'txHash': '0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d92b0e1512a81b1129e3

**Transform your data format**

In [12]:
with open("data.json", "r") as f:
    raw_txns = json.load(f)

wallet_map = {}

for txn in raw_txns:
    wallet = txn.get("userWallet")
    timestamp = txn.get("timestamp")
    action = txn.get("action")
    amount = txn.get("actionData", {}).get("amount", "0")

    if wallet and timestamp and action:
        if wallet not in wallet_map:
            wallet_map[wallet] = []

        wallet_map[wallet].append({
            "action": action.lower(),
            "amount": float(amount),
            "timestamp": int(timestamp)
        })


**Compute Features and Scores**

In [13]:
features = [compute_features(wallet, txns) for wallet, txns in wallet_map.items()]
df = pd.DataFrame(features)
df["credit_score"] = df.apply(score_wallet, axis=1)
df[["wallet", "credit_score"]].sort_values(by="credit_score", ascending=False).head(10)


,wallet,credit_score
614,0x00f656a2953a15ddf8d9d9726b67dd4e61f238b9,1000
584,0x00e6f8d8fb80b0c302a6bd849b79982dc9945b15,1000
1790,0x030888e8fe9f0cbe3ff04c0fda1c0954c6b68822,1000
629,0x00fbef9f0f9b0b6421c264a2020f14bca80291ca,1000
3131,0x057536f8e7ae97f17aa5fd21bd7942a7fe152241,1000
1802,0x030d9f55c7f7589034b829a87bf3bd84d6682a18,1000
1058,0x01b92f893b1c302bcf1945fd4edf10df99b4f181,1000
1850,0x03247269bad6aa35ef79ff7f674d714c64d26611,1000
32,0x0005696d4be2359526c497fe4a4c0620b41f2426,1000
53,0x000f4432a40560bbff1b581a8b7aded8dab80026,1000


**Export Results**

In [14]:
df[["wallet", "credit_score"]].to_csv("wallet_scores.csv", index=False)
files.download("wallet_scores.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>